In [ ]:
import numpy as np
import cv2
import sys
import os
import shutil
import pandas as pd

In [ ]:
# assume Nexar dataset is in 'data' folder
data_root = os.path.abspath('./data')
train_path = os.path.join(data_root, 'train')
train_desc_path = os.path.join(data_root, 'train.csv')
train_boxes_path = os.path.join(data_root, 'train_boxes.csv')

train_img_paths = os.listdir(train_path)
num_train = len(train_img_paths)

print 'data root path: ', data_root
print 'train path: ', train_path
print 'train examples: ', num_train

In [ ]:
names_df = pd.read_csv(train_desc_path)
labels_df = pd.read_csv(train_boxes_path)

labeled_image_names = list(set(labels_df['image_filename']))
missing_image_names = list(set(names_df['image_filename']).difference(labeled_image_names))
car_types = list(set(labels_df['label']))

print '### train.csv ###'
print 'shape: ', names_df.shape
print names_df.head()
print '\n### train_boxes.csv ###'
print 'shape: ', labels_df.shape
print labels_df.head()
print 
print 'car types: ', car_types
print 'missing images: ', len(missing_image_names)

labels_df.set_index('image_filename', inplace=True)

In [ ]:
# randomly select N examples
num_rnd_examples = 100
rnd_examples_names = np.random.choice(labeled_image_names, num_rnd_examples)

colors = {'pickup_truck': (255, 255, 0),
          'car': (255, 0, 0),
          'van': (0, 255, 0),
          'truck': (0, 0, 255),
          'bus': (0, 255, 255)}

def draw_legend(img):
    for i, t in enumerate(colors.keys()):
        org = (20, 12 + i * 10)
        res = cv2.putText(img, t, org, cv2.FONT_HERSHEY_PLAIN, 1, colors[t], 1)
    return res

def draw_box(img, x0, y0, x1, y1, label):
    x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
    res = cv2.rectangle(img, (x0, y0), (x1, y1), colors[label], 2)
    return res

# draw boxes on random images and save
save_dir = './random_examples'
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)
os.makedirs(save_dir)

for fname in rnd_examples_names:
    full_path = os.path.join(train_path, fname)
    img = cv2.imread(full_path)
    
    df = labels_df.loc[fname]
    if df.ndim > 1: # multiple boxes
        for row in df.itertuples():
            img = draw_box(img, row.x0, row.y0, row.x1, row.y1, row.label)
    else: # one box
        img = draw_box(img, df.x0, df.y0, df.x1, df.y1, df.label)
        
    img = draw_legend(img)
        
    cv2.imwrite(os.path.join(save_dir, fname), img)